In [1]:
import numpy as np
#import pandas as pd
import pickle
import math
import os
import matplotlib.pyplot as plt
from utils import get_cableidx
import collections
###################
n_f=4
n_node=106
time_step=1
###################
#label - load fact
y_lf=np.load('data/loadfact_list.npy').reshape((-1,1))#(2960, 10)
y_lf=y_lf*700000
###node ddata 
#location feature
node_loc=np.load('data/node_loc.npy')
x=np.array([node_loc]*74*40)
#load feature
time_step=3
x_offsets = np.sort(np.concatenate((np.arange(0, time_step, 1),)))
load_loc=[]
idx_node=np.arange(1,39+1)
for t in range(0,len(idx_node)-time_step+1):
    load_loc.append(idx_node[t + x_offsets, ...])
idx_node=np.arange(40,78+1)
for t in range(0,len(idx_node)-time_step+1):
    load_loc.append(idx_node[t + x_offsets, ...])
load_loc=np.array(load_loc)
load_loc=load_loc-1
load_loc=np.repeat(load_loc, repeats=40, axis=0)
load_node=np.zeros((74*40,106))
for i, c in enumerate(load_loc): load_node[i,c]=1
x=np.concatenate((x,load_node[:,:,np.newaxis]),axis=-1);print(x.shape)#x,y,z,vehicle loc
#############################
features=x.copy()
label_lf=y_lf.copy()
#split
np.random.seed(1)
indices = np.random.permutation(len(features));print(indices[0])
n_training=int(len(features)*0.6)
n_val=int(len(features)*0.2)
n_test=len(features)-n_training-n_val
training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
f_training, f_val, f_test = features[training_idx],features[val_idx], features[test_idx]
print(f_training.shape,f_val.shape,f_test.shape)
l_training, l_val, l_test = label_lf[training_idx],label_lf[val_idx], label_lf[test_idx]
print(l_training.shape,l_val.shape,l_test.shape)

#scale-features
for i in range(n_f-1):
    m=np.mean(f_training[...,i].flatten());std=np.std(f_training[...,i].flatten())
    f_training[...,i]=(f_training[...,i]-m)/std
    f_val[...,i]=(f_val[...,i]-m)/std
    f_test[...,i]=(f_test[...,i]-m)/std
for i in range(n_f): print(i,np.min(f_val[...,i]),np.max(f_val[...,i]),np.mean(f_val[...,i]),np.std(f_val[...,i]))
#scale-label
m=np.mean(l_training);std=np.std(l_training)
l_training=(l_training-m)/std
l_val=(l_val-m)/std
l_test=(l_test-m)/std
print('label m, std',m,",",std)
print('label',np.min(l_training),np.max(l_training),np.mean(l_training),np.std(l_training))

#small graph
# in_cable=np.array([1,4,9,11,16,21,24,29,31,36])-1
# node_cable= (get_cableidx()[in_cable,2])-1
# node_cable=np.concatenate((np.arange(78),node_cable),axis=0)

save_flag=False#True#
if save_flag:
    data_path='data/data_mpnn/'
    np.savez_compressed(data_path+'features_train',f_training)
    np.savez_compressed(data_path+'features_val',f_val)
    np.savez_compressed(data_path+'features_test',f_test)
    data_path='data/'
    np.savez_compressed(data_path+'label_train',l_training)
    np.savez_compressed(data_path+'label_val',l_val)
    np.savez_compressed(data_path+'label_test',l_test)
#     data_path='data/data_mpnn_simple/'
#     np.savez_compressed(data_path+'features_train',f_training[:,:,3:])
#     np.savez_compressed(data_path+'features_val',f_val[:,:,3:])
#     np.savez_compressed(data_path+'features_test',f_test[:,:,3:])
#     data_path='data/data_mpnn_simple_dist/'
#     np.savez_compressed(data_path+'features_train',f_training[:,:,3:])
#     np.savez_compressed(data_path+'features_val',f_val[:,:,3:])
#     np.savez_compressed(data_path+'features_test',f_test[:,:,3:])
#     data_path='data/data_mpnn_small/'
#     np.savez_compressed(data_path+'features_train',f_training[:,node_cable,:])
#     np.savez_compressed(data_path+'features_val',f_val[:,node_cable,:])
#     np.savez_compressed(data_path+'features_test',f_test[:,node_cable,:])

(2960, 106, 4)
796
(1776, 106, 4) (592, 106, 4) (592, 106, 4)
(1776, 1) (592, 1) (592, 1)
0 -1.7371883443796705 1.7371883443796705 -1.9390687706992154e-18 1.0
1 -1.0 1.0 0.0 1.0
2 -1.4582693232632356 2.5671616211613206 3.2723554728880185e-17 1.0
3 0.0 1.0 0.02830188679245283 0.16583392293629198
label m, std 355311.0926130971 , 201425.32593642248
label -1.6771033671255067 1.7112490983169608 3.800763507725761e-17 0.9999999999999999


In [1]:
import numpy as np
import pickle
import collections
from utils import get_cableidx

with open("data/sensor_graph/adj_mx.pkl", 'rb') as pickle_file:
    adj_mx = pickle.load(pickle_file)
for i in range(106): adj_mx[i,i]=0
#delete 50 node#adj_mx[out_node][:,out_node]=0
print(adj_mx.shape)
edge_data=[[i,j] for i,j in zip(np.where(adj_mx==1)[0],np.where(adj_mx==1)[1])];edge_data=np.array(edge_data)
edge_data=edge_data[edge_data[:,0]<edge_data[:,1]]

#distance & E
with open("data/sensor_graph/adj_mx_type_e.pkl", 'rb') as pickle_file:
    edge_de = pickle.load(pickle_file)#sacled [distance, E]
edge_d=edge_de[2][0];edge_e=edge_de[2][1]
print(collections.Counter(edge_d.flatten()))
print(collections.Counter(edge_e.flatten()))
for i in range(106): edge_d[i,i]=0
for i in range(106): edge_e[i,i]=0
#area
with open("data/sensor_graph/adj_mx_type_a.pkl", 'rb') as pickle_file:
    edge_a = pickle.load(pickle_file)
print(collections.Counter(edge_a.flatten()))
for i in range(106): edge_a[i,i]=0

#make inputs for graph
edge_a=edge_a[edge_data[:,0],edge_data[:,1]]
edge_d=edge_d[edge_data[:,0],edge_data[:,1]]
edge_e=edge_e[edge_data[:,0],edge_data[:,1]]
edge_f=np.stack([edge_d,edge_e,edge_a],axis=1)
print(edge_data.shape,edge_f.shape)#(149, 106, 6) (149, 40) (185, 2) (185, 2)

#self loop
edge_self=np.array([[i,i] for i in range(len(adj_mx))])
edge_data=np.concatenate([edge_data,edge_self],axis=0)
edge_f=np.concatenate([edge_f,np.zeros((len(adj_mx),edge_f.shape[1]))],axis=0)#matching shape (below)
edge_f_self=np.array([0]*(len(edge_f)-len(adj_mx))+[1]*(len(adj_mx)))[:,np.newaxis]#self loop feature
edge_f=np.concatenate([edge_f,edge_f_self],axis=1)
print(edge_data.shape,edge_f.shape)

#########tension data
in_cable=np.array([1,4,9,11,16,21,24,29,31,36])-1#list(range(3,40,4))#[3,7,11,15,19,23,27,31,35,39]
tension=np.load('data/efo_signal_list.npy')#74,40,40 #case, step, cable 

#mlp label data -tension 30
'''out_cable=np.array(list(set(range(40))-set(in_cable)))
tension_30=tension[:,:,out_cable].reshape((-1,30));print(tension_30.shape)'''

#edge data - tension 40
tension_40=tension.reshape((-1,40))
node_cable_40=(get_cableidx()[:,1:])-1
edge_t_40=np.zeros((74*40,291,1))
for i in range(74*40):
    for j,(r,c) in enumerate(node_cable_40): 
        idx=np.where((edge_data == (r, c)).all(axis=1))
        edge_t_40[i,idx,0]=tension_40[i,j]
edge_f_40=np.array([edge_f]*74*40)
edge_f_40=np.concatenate((edge_f_40,edge_t_40),axis=-1);print(edge_f_40.shape)#distance, E, area, self, tension

#edge data & mlp label data - tension 10
node_cable= (get_cableidx()[in_cable,1:])-1
tension=tension[:,:,in_cable].reshape((-1,10))#mlp input data
edge_t=np.zeros((74*40,291,1))
for i in range(74*40):
    for j,(r,c) in enumerate(node_cable): 
        idx=np.where((edge_data == (r, c)).all(axis=1))
        edge_t[i,idx,0]=tension[i,j]
edge_f=np.array([edge_f]*74*40)
edge_f=np.concatenate((edge_f,edge_t),axis=-1);print(edge_f.shape)#distance, E, area, self, tension

#indexing
np.random.seed(1)
indices = np.random.permutation(len(edge_f));print(indices[0])
n_training=int(len(edge_f)*0.6)#7:1:2
n_val=int(len(edge_f)*0.2)
n_test=len(edge_f)-n_training-n_val
training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]

e_training, e_val,e_test = edge_f[training_idx],edge_f[val_idx], edge_f[test_idx]
e_training_40, e_val_40,e_test_40 = edge_f_40[training_idx],edge_f_40[val_idx], edge_f_40[test_idx]
'''mlp_training, mlp_val,mlp_test = tension[training_idx],tension[val_idx], tension[test_idx]#mlp input data
mlp_label_training, mlp_label_val,mlp_label_test = tension_30[training_idx],tension_30[val_idx], tension_30[test_idx]'''#mlp input data
print(e_training.shape,e_val.shape,e_test.shape)

###scale
for i in [0,1,2,4]:
    m=np.mean(e_training[...,i].flatten());std=np.std(e_training[...,i].flatten())
    e_training[...,i]=(e_training[...,i]-m)/std
    e_val[...,i]=(e_val[...,i]-m)/std
    e_test[...,i]=(e_test[...,i]-m)/std
for i in range(5): print('10',i,np.min(e_val[...,i]),np.max(e_val[...,i]),np.mean(e_val[...,i]),np.std(e_val[...,i]))
for i in [0,1,2,4]:
    m=np.mean(e_training_40[...,i].flatten());std=np.std(e_training_40[...,i].flatten())
    e_training_40[...,i]=(e_training_40[...,i]-m)/std
    e_val_40[...,i]=(e_val_40[...,i]-m)/std
    e_test_40[...,i]=(e_test_40[...,i]-m)/std
for i in range(5): print('40',i,np.min(e_val_40[...,i]),np.max(e_val_40[...,i]),np.mean(e_val_40[...,i]),np.std(e_val_40[...,i]))
#mlp
'''m=np.mean(mlp_training.flatten());std=np.std(mlp_training.flatten())
mlp_training=(mlp_training-m)/std;mlp_val=(mlp_val-m)/std;mlp_test=(mlp_test-m)/std
print('mlp',np.min(mlp_val),np.max(e_val),np.mean(e_val),np.std(e_val))
m=np.mean(mlp_label_training.flatten());std=np.std(mlp_label_training.flatten())
mlp_label_training=(mlp_label_training-m)/std;mlp_label_val=(mlp_label_val-m)/std;mlp_label_test=(mlp_label_test-m)/std'''

save_flag=True#False#
if save_flag:
    data_path='data/data_mpnn/'
    np.savez_compressed(data_path+'edge_train',e_training)
    np.savez_compressed(data_path+'edge_val',e_val)
    np.savez_compressed(data_path+'edge_test',e_test)
    np.savez_compressed(data_path+'edge_data',edge_data)
    
    data_path='data/data_mpnn/tension40/'
    np.savez_compressed(data_path+'edge_train',e_training_40)
    np.savez_compressed(data_path+'edge_val',e_val_40)
    np.savez_compressed(data_path+'edge_test',e_test_40)
    np.savez_compressed(data_path+'edge_data',edge_data)
    
    '''data_path='data/data_mlp/'
    np.savez_compressed(data_path+'features_train',mlp_training)
    np.savez_compressed(data_path+'features_val',mlp_val)
    np.savez_compressed(data_path+'features_test',mlp_test)
    
    data_path='data/data_mlp/data_tension30/'
    np.savez_compressed(data_path+'label_train',mlp_label_training)
    np.savez_compressed(data_path+'label_val',mlp_label_val)
    np.savez_compressed(data_path+'label_test',mlp_label_test)'''
    
#     data_path='data/data_mpnn_simple/'
#     np.savez_compressed(data_path+'edge_train',e_training[:,:,-1:])
#     np.savez_compressed(data_path+'edge_val',e_val[:,:,-1:])
#     np.savez_compressed(data_path+'edge_test',e_test[:,:,-1:])
#     np.savez_compressed(data_path+'edge_data',edge_data)
#     data_path='data/data_mpnn_simple_dist/'
#     np.savez_compressed(data_path+'edge_train',e_training[:,:,[0,-1]])
#     np.savez_compressed(data_path+'edge_val',e_val[:,:,[0,-1]])
#     np.savez_compressed(data_path+'edge_test',e_test[:,:,[0,-1]])
#     np.savez_compressed(data_path+'edge_data',edge_data)

(106, 106)
Counter({0.0: 10866, 6100.0: 152, 7500.0: 82, 3000.0: 32, 62562.05078125: 16, 50520.1953125: 16, 38810.4375: 16, 27854.802734375: 16, 19005.525390625: 16, 9000.0: 16, 10000.0: 8})
Counter({0.0: 10866, 200000.0: 290, 15800.0: 80})
Counter({0.0: 10866, 97500.0: 152, 3848.45: 80, 8245.14480000001: 78, 147500.0: 60})
(185, 2) (185, 3)
(291, 2) (291, 4)
(2960, 291, 5)
(2960, 291, 5)
796
(1776, 291, 5) (592, 291, 5) (592, 291, 5)
10 0 -0.6170756195935757 3.8389546314661023 1.099189880422046e-17 0.9999999999999999
10 1 -1.03946313647513 1.0021390011751319 2.5035957126310765e-17 0.9999999999999999
10 2 -0.7792581200288932 1.9377485273458475 -3.086805652937558e-16 0.9999999999999997
10 3 0.0 1.0 0.3642611683848797 0.4812223702112797
10 4 -0.17046215567487813 10.634464829382013 0.0005318686838786754 1.0035485464922675
40 0 -0.6170756195935757 3.8389546314661023 1.099189880422046e-17 0.9999999999999999
40 1 -1.03946313647513 1.0021390011751319 2.5035957126310765e-17 0.9999999999999999
